In [2]:
# Run this cell in case environment needs to be set up - uncomment below lines
#!pip install --upgrade opencv-python
#!pip install --upgrade watson-developer-cloud

In [16]:
#Cleaning the local machine
#!ls
!rm *.*
!ls

data


In [17]:
# The code was removed by DSX for sharing.

File Downloaded
File terminator1984.mp4 downloaded to ntv.mp4


In [18]:
import cv2
import numpy as np
import os
import zipfile
import re
import json
from io import BytesIO
from os.path import join, dirname
from os import environ
from watson_developer_cloud import VisualRecognitionV3
import pandas as pd
from pandas.io.json import json_normalize
import seaborn as sns
from matplotlib import pyplot
from scipy.cluster import hierarchy
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import AgglomerativeClustering


In [19]:
!ls -all
!pwd
data_dir = './data/'
cap = cv2.VideoCapture(local_name)
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#visual_recognition = VisualRecognitionV3('2018-06-07', api_key='SaKFXYE6-XKhVcARkBCtZ6OuB_l9m3aZemHG_ihYsZOG')
visual_recognition = VisualRecognitionV3('2018-06-07', iam_api_key='XsGGHdWrBo8jaJkNIGA9aSr6_QXvYZNH5xi58zkLRnGa')
print(cap.open(local_name))
try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print ('Error: Creating directory of data')

total 7808
drwxr-x--- 3 dsxuser dsxuser    4096 Jul 23 12:27 .
drwx------ 1 dsxuser dsxuser    4096 Jul 23 11:18 ..
drwxr-x--- 2 dsxuser dsxuser    4096 Jul 23 11:19 data
-rw-r----- 1 dsxuser dsxuser 7979254 Jul 23 12:27 ntv.mp4
/home/dsxuser/work
2841.0
True


In [20]:
def getClasses(json_data):
    return dict(list(map( lambda el: [el['class'],el['score']],json_data["images"][0]["classifiers"][0]["classes"])))

def getComplexClasses(json_data):
    for image in json_data['images']:
        res = re.match(r'.*frame(\d*)',image['image'], re.M|re.I)
        print(res.group(1))
        print(dict(list(map( lambda el: [el['class'],el['score']],image["classifiers"][0]["classes"]))))


def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            print(file)
            ziph.write(os.path.join(root, file))

In [21]:
currentFrame = 0
frames_dict = []
frames_to_pass=100
print("Hey")
ret, frame = cap.read()
with open('result_dict.json', 'w') as res_file:    
    print('test')
while(frame is not None):
    if(currentFrame%frames_to_pass):
        currentFrame += 1
        ret, frame = cap.read()
        continue
    if(not currentFrame%10):
        print(currentFrame)
    name = data_dir + 'frame0.jpg'
#     print ('Analyzing...' + str(currentFrame))
    cv2.imwrite(name, frame)
    ret, buf = cv2.imencode('.jpg',frame)
#     print(BytesIO(frame))
    #print(type(buf))
    with open(data_dir + 'frame0.jpg', 'rb') as image_file:
#         print(image_file)
        json_data = visual_recognition.classify(images_file=image_file, threshold=0, classifier_ids=['default'])
        currElement = {'id':currentFrame}
        currElement.update(getClasses(json_data))
        frames_dict.append(currElement)
    #print(frames_dict)
    ret, frame = cap.read()

    currentFrame += 1
    
with open('result_dict.json', 'w') as res_file: 
    res_file.write(json.dumps(frames_dict))
df = json_normalize(frames_dict)


Hey
test
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [22]:
cap.release()
cv2.destroyAllWindows()

In [23]:
df = df.fillna(0)
df.head()

,Alley,Clubbing,Fleet Street building,Guinness,Indian red color,Lightning,Night Sky,Riot,Sheetrock,Teleprompter,...,vigil light,wainscoting wood panels,wall,wall unit,weapon,web site,welder's mask,window,window frame,woman
0,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.704,0.0,0.000,0.583,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000,0.0,0.557,0.811,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.616,0.0,0.0,0.0


In [24]:
#frames_to_pass = 100
local_csv_name = "out_ntv_"+str(frames_to_pass)+".csv"
df.to_csv(local_csv_name)

In [25]:
df = pd.read_csv(local_csv_name)
df.head()

,Unnamed: 0,Alley,Clubbing,Fleet Street building,Guinness,Indian red color,Lightning,Night Sky,Riot,Sheetrock,...,vigil light,wainscoting wood panels,wall,wall unit,weapon,web site,welder's mask,window,window frame,woman
0,0,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.704,0.0,0.000,0.583,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.000,0.0,0.557,0.811,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.616,0.0,0.0,0.0


In [26]:
cos_csv_name = video_name+"_"+str(frames_to_pass)+".csv"
upload_file_cos(credentials,local_csv_name,cos_csv_name)
print ("File "+local_csv_name+" uploaded to "+cos_csv_name)

 File Uploaded
File out_ntv_100.csv uploaded to terminator1984_100.csv


In [15]:
no_id = df.drop(['id','Unnamed: 0'], axis=1)
# no_id = df.drop(['id'], axis=1)
# no_id = df.drop(['Unnamed: 0'], axis=1)